In [ ]:
#!/usr/bin/env python
# coding: utf-8

import os
import csv
import time
import math
import scipy
import xgboost as xgb
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std
from numpy import genfromtxt

import pickle
import logging

import glob
import statistics
import statsmodels.api as sm
from statistics import mean
from statistics import stdev
from pytz import timezone
from datetime import datetime
from scipy.stats import shapiro
from scipy.signal import resample
from scipy.cluster.vq import kmeans
from xgboost import plot_importance
from matplotlib import pyplot

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import r2_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report

# Prepare Dataset

In [ ]:
# df_shape = pd.read_csv(f'./_results/norotate_looc_384.csv', skiprows = 0)  # LOOC
df_shape = pd.read_csv(f'./_results/df_512_mask_all_210927.csv', skiprows = 0)  # Ours FINAL!

In [ ]:
df_mask_columns = []

for i in range(0,512):
    df_mask_columns.append("all_"+str(i))
for i in range(0,512):
    df_mask_columns.append("shape_"+str(i))    
for i in range(0,512):
    df_mask_columns.append("color_"+str(i))  

In [ ]:
df_shape_mid1 = pd.concat([df_mask_all.iloc[:, :512], df_mask_shape.iloc[:, :512]], axis=1)
df_shape_mid2 = pd.concat([df_shape_mid1, df_mask_color.iloc[:, :512]], axis=1)

df_shape_mid2.columns = df_mask_columns
df_shape_mid2.shape

In [ ]:
df_shape = pd.concat([df_shape_mid2, df_mask_all.iloc[:, 512:]], axis=1)
df_shape.shape

In [ ]:
shape_dim = 512*3  # 384; 512; 512*3
df_shape_only = pd.concat([df_shape.iloc[:, shape_dim], df_shape.iloc[:, 0:shape_dim]], axis=1)
df_shape_only.shape

In [ ]:
## Feature-engineered Features

bins_no = 128  # = 32, 128, 256

In [ ]:
# df_col_seg_meta_raw = pd.concat([df_shape.iloc[:, shape_dim], df_shape.iloc[:, shape_dim+1:]], axis=1)

In [ ]:
with open(f'./_results/total_df_RGBHSV_ent_seg_rgbHistBin{bins_no}_meta.pkl', 'rb') as f:
    object_df = pickle.load(f)
    
df_col_seg_meta_raw = pd.DataFrame(object_df)

In [ ]:
## [START] Y-label: gender

print(np.unique(df_col_seg_meta_raw['gender'].values))
len(np.unique(df_col_seg_meta_raw['gender'].values))

In [ ]:
df_col_seg_meta_raw['gender'].value_counts()

In [ ]:
df_col_seg_meta_mid = df_col_seg_meta_raw.loc[(df_col_seg_meta_raw['gender'] == 'child') | 
                                              (df_col_seg_meta_raw['gender'] == 'preschool') | 
                                              (df_col_seg_meta_raw['gender'] == 'toddler') |
                                              (df_col_seg_meta_raw['gender'] == 'women')]

In [ ]:
import random

men_list = df_col_seg_meta_raw.loc[df_col_seg_meta_raw['gender'] == "men"].index.values.tolist()
sample_random = random.sample(men_list, 3000)

In [ ]:
df_men_only = df_col_seg_meta_raw.loc[sample_random]

In [ ]:
df_col_seg_meta = pd.concat([df_men_only, df_col_seg_meta_mid])

In [ ]:
df_col_seg_meta.shape

In [ ]:
df_col_seg_meta['gender'].value_counts()

In [ ]:
df_col_seg_meta.columns.values

In [ ]:
comparison_column = np.where(df_col_seg_meta["image_fileName"] == df_col_seg_meta["id"], True, False)

m=0
for x in comparison_column:
    if x == False:
        m += 1
print(m)

## [END] Y-label: gender

In [ ]:
## [START] Y-label: primaryCategory

print(np.unique(df_col_seg_meta_raw['primaryCategory'].values))
len(np.unique(df_col_seg_meta_raw['primaryCategory'].values))

In [ ]:
df_col_seg_meta_raw['primaryCategory'].value_counts()

In [ ]:
df_col_seg_meta = df_col_seg_meta_raw

In [ ]:
df_col_seg_meta = df_col_seg_meta_raw.loc[(df_col_seg_meta_raw['primaryCategory'] != 'SPO') & (df_col_seg_meta_raw['primaryCategory'] != 'Nike Training') &
                                          (df_col_seg_meta_raw['primaryCategory'] != 'Nike Running') & (df_col_seg_meta_raw['primaryCategory'] != 'Luxury Brands') &
                                          (df_col_seg_meta_raw['primaryCategory'] != 'Foamposite') & (df_col_seg_meta_raw['primaryCategory'] != 'KD') &
                                          (df_col_seg_meta_raw['primaryCategory'] != 'Nike Other') & (df_col_seg_meta_raw['primaryCategory'] != 'Other Brands')]

In [ ]:
print(df_col_seg_meta['primaryCategory'].value_counts())
len(df_col_seg_meta['primaryCategory'])

In [ ]:
df_col_seg_meta.shape

## [END] Y-label: primaryCategory

In [ ]:
## [START] Y-label: maxResellPrice

df_maxResell = pd.read_csv(f'./_results/max_resell_prices_Bre_210917.csv', skiprows = 0)

In [ ]:
i = 0
for val in df_maxResell:
    print(i, val)
    i += 1
    
print('\n', "Total # of features:", str(i))

In [ ]:
df_maxResell_premium = df_maxResell.iloc[:, 7:]

In [ ]:
df_maxResell_premium.head()

In [ ]:
df_maxResell_premium.shape

In [ ]:
import matplotlib.pyplot as plt

plt.hist(df_maxResell_premium['inflated_profit'], bins=100, label='bins=100')
plt.legend()
plt.show()

In [ ]:
# QQ Plot
from numpy.random import seed
from numpy.random import randn
from statsmodels.graphics.gofplots import qqplot
from matplotlib import pyplot

# seed the random number generator
seed(1)
# generate univariate observations
data = 5 * randn(100) + 50
qqplot(data, line='s')
pyplot.show()

# q-q plot
qqplot(df_maxResell_premium['inflated_profit'], line='s')
pyplot.show()

In [ ]:
stat, p = shapiro(df_maxResell_premium['inflated_profit'])
print('Statistics=%.3f, p=%.3f' % (stat, p))

# interpret
alpha = 0.05
if p > alpha:
    print('Sample looks Gaussian (fail to reject H0)')
else:
    print('Sample does not look Gaussian (reject H0)')

In [ ]:
print("20th percentile: ", np.percentile(df_maxResell_premium['inflated_profit'], 20))
print("25th percentile: ", np.percentile(df_maxResell_premium['inflated_profit'], 25))
print("50th percentile: ", np.percentile(df_maxResell_premium['inflated_profit'], 50))
print("75th percentile: ", np.percentile(df_maxResell_premium['inflated_profit'], 75))
print("80th percentile: ", np.percentile(df_maxResell_premium['inflated_profit'], 80))

pct_20 = np.percentile(df_maxResell_premium['inflated_profit'], 20)
pct_80 = np.percentile(df_maxResell_premium['inflated_profit'], 80)

In [ ]:
df_mid_premium_lower = df_maxResell_premium.loc[(df_maxResell_premium['inflated_profit']< pct_80)]
df_bin_premium_upper = df_maxResell_premium.loc[(df_maxResell_premium['inflated_profit']>=pct_80)]

In [ ]:
df_bin_premium_lower = df_mid_premium_lower

In [ ]:
import random

premium_lower_list = df_mid_premium_lower.index.values.tolist()
premium_lower_sample = random.sample(premium_lower_list, len(df_bin_premium_upper))

In [ ]:
df_bin_premium_lower = df_mid_premium_lower[df_mid_premium_lower.index.isin(premium_lower_sample)]

In [ ]:
df_bin_premium_lower['bin_premium'] = "premium_lower"
df_bin_premium_upper['bin_premium'] = "premium_upper"

In [ ]:
df_bin_premium = pd.concat([df_bin_premium_lower, df_bin_premium_upper])

In [ ]:
df_bin_premium

In [ ]:
## [END] Y-label: maxResellPrice

In [ ]:
# normalize by z-score by column

col_zscore = ['mean_b', 'mean_g', 'mean_r', 'std_b', 'std_g',
       'std_r', 'mean_h', 'mean_s', 'mean_v', 'std_h', 'std_s', 'std_v',
       'ent_b', 'ent_g', 'ent_r', 'ent_h', 'ent_s', 'ent_v', 'ent_grey',
       'mean_area', 'std_area', 'mean_perimeter', 'std_perimeter',
       'num_segments', 'B_hist_0', 'B_hist_1', 'B_hist_2', 'B_hist_3',
       'B_hist_4', 'B_hist_5', 'B_hist_6', 'B_hist_7', 'B_hist_8',
       'B_hist_9', 'B_hist_10', 'B_hist_11', 'B_hist_12', 'B_hist_13',
       'B_hist_14', 'B_hist_15', 'B_hist_16', 'B_hist_17', 'B_hist_18',
       'B_hist_19', 'B_hist_20', 'B_hist_21', 'B_hist_22', 'B_hist_23',
       'B_hist_24', 'B_hist_25', 'B_hist_26', 'B_hist_27', 'B_hist_28',
       'B_hist_29', 'B_hist_30', 'B_hist_31', 'B_hist_32', 'B_hist_33',
       'B_hist_34', 'B_hist_35', 'B_hist_36', 'B_hist_37', 'B_hist_38',
       'B_hist_39', 'B_hist_40', 'B_hist_41', 'B_hist_42', 'B_hist_43',
       'B_hist_44', 'B_hist_45', 'B_hist_46', 'B_hist_47', 'B_hist_48',
       'B_hist_49', 'B_hist_50', 'B_hist_51', 'B_hist_52', 'B_hist_53',
       'B_hist_54', 'B_hist_55', 'B_hist_56', 'B_hist_57', 'B_hist_58',
       'B_hist_59', 'B_hist_60', 'B_hist_61', 'B_hist_62', 'B_hist_63',
       'B_hist_64', 'B_hist_65', 'B_hist_66', 'B_hist_67', 'B_hist_68',
       'B_hist_69', 'B_hist_70', 'B_hist_71', 'B_hist_72', 'B_hist_73',
       'B_hist_74', 'B_hist_75', 'B_hist_76', 'B_hist_77', 'B_hist_78',
       'B_hist_79', 'B_hist_80', 'B_hist_81', 'B_hist_82', 'B_hist_83',
       'B_hist_84', 'B_hist_85', 'B_hist_86', 'B_hist_87', 'B_hist_88',
       'B_hist_89', 'B_hist_90', 'B_hist_91', 'B_hist_92', 'B_hist_93',
       'B_hist_94', 'B_hist_95', 'B_hist_96', 'B_hist_97', 'B_hist_98',
       'B_hist_99', 'B_hist_100', 'B_hist_101', 'B_hist_102',
       'B_hist_103', 'B_hist_104', 'B_hist_105', 'B_hist_106',
       'B_hist_107', 'B_hist_108', 'B_hist_109', 'B_hist_110',
       'B_hist_111', 'B_hist_112', 'B_hist_113', 'B_hist_114',
       'B_hist_115', 'B_hist_116', 'B_hist_117', 'B_hist_118',
       'B_hist_119', 'B_hist_120', 'B_hist_121', 'B_hist_122',
       'B_hist_123', 'B_hist_124', 'B_hist_125', 'B_hist_126',
       'B_hist_127', 'G_hist_0', 'G_hist_1', 'G_hist_2', 'G_hist_3',
       'G_hist_4', 'G_hist_5', 'G_hist_6', 'G_hist_7', 'G_hist_8',
       'G_hist_9', 'G_hist_10', 'G_hist_11', 'G_hist_12', 'G_hist_13',
       'G_hist_14', 'G_hist_15', 'G_hist_16', 'G_hist_17', 'G_hist_18',
       'G_hist_19', 'G_hist_20', 'G_hist_21', 'G_hist_22', 'G_hist_23',
       'G_hist_24', 'G_hist_25', 'G_hist_26', 'G_hist_27', 'G_hist_28',
       'G_hist_29', 'G_hist_30', 'G_hist_31', 'G_hist_32', 'G_hist_33',
       'G_hist_34', 'G_hist_35', 'G_hist_36', 'G_hist_37', 'G_hist_38',
       'G_hist_39', 'G_hist_40', 'G_hist_41', 'G_hist_42', 'G_hist_43',
       'G_hist_44', 'G_hist_45', 'G_hist_46', 'G_hist_47', 'G_hist_48',
       'G_hist_49', 'G_hist_50', 'G_hist_51', 'G_hist_52', 'G_hist_53',
       'G_hist_54', 'G_hist_55', 'G_hist_56', 'G_hist_57', 'G_hist_58',
       'G_hist_59', 'G_hist_60', 'G_hist_61', 'G_hist_62', 'G_hist_63',
       'G_hist_64', 'G_hist_65', 'G_hist_66', 'G_hist_67', 'G_hist_68',
       'G_hist_69', 'G_hist_70', 'G_hist_71', 'G_hist_72', 'G_hist_73',
       'G_hist_74', 'G_hist_75', 'G_hist_76', 'G_hist_77', 'G_hist_78',
       'G_hist_79', 'G_hist_80', 'G_hist_81', 'G_hist_82', 'G_hist_83',
       'G_hist_84', 'G_hist_85', 'G_hist_86', 'G_hist_87', 'G_hist_88',
       'G_hist_89', 'G_hist_90', 'G_hist_91', 'G_hist_92', 'G_hist_93',
       'G_hist_94', 'G_hist_95', 'G_hist_96', 'G_hist_97', 'G_hist_98',
       'G_hist_99', 'G_hist_100', 'G_hist_101', 'G_hist_102',
       'G_hist_103', 'G_hist_104', 'G_hist_105', 'G_hist_106',
       'G_hist_107', 'G_hist_108', 'G_hist_109', 'G_hist_110',
       'G_hist_111', 'G_hist_112', 'G_hist_113', 'G_hist_114',
       'G_hist_115', 'G_hist_116', 'G_hist_117', 'G_hist_118',
       'G_hist_119', 'G_hist_120', 'G_hist_121', 'G_hist_122',
       'G_hist_123', 'G_hist_124', 'G_hist_125', 'G_hist_126',
       'G_hist_127', 'R_hist_0', 'R_hist_1', 'R_hist_2', 'R_hist_3',
       'R_hist_4', 'R_hist_5', 'R_hist_6', 'R_hist_7', 'R_hist_8',
       'R_hist_9', 'R_hist_10', 'R_hist_11', 'R_hist_12', 'R_hist_13',
       'R_hist_14', 'R_hist_15', 'R_hist_16', 'R_hist_17', 'R_hist_18',
       'R_hist_19', 'R_hist_20', 'R_hist_21', 'R_hist_22', 'R_hist_23',
       'R_hist_24', 'R_hist_25', 'R_hist_26', 'R_hist_27', 'R_hist_28',
       'R_hist_29', 'R_hist_30', 'R_hist_31', 'R_hist_32', 'R_hist_33',
       'R_hist_34', 'R_hist_35', 'R_hist_36', 'R_hist_37', 'R_hist_38',
       'R_hist_39', 'R_hist_40', 'R_hist_41', 'R_hist_42', 'R_hist_43',
       'R_hist_44', 'R_hist_45', 'R_hist_46', 'R_hist_47', 'R_hist_48',
       'R_hist_49', 'R_hist_50', 'R_hist_51', 'R_hist_52', 'R_hist_53',
       'R_hist_54', 'R_hist_55', 'R_hist_56', 'R_hist_57', 'R_hist_58',
       'R_hist_59', 'R_hist_60', 'R_hist_61', 'R_hist_62', 'R_hist_63',
       'R_hist_64', 'R_hist_65', 'R_hist_66', 'R_hist_67', 'R_hist_68',
       'R_hist_69', 'R_hist_70', 'R_hist_71', 'R_hist_72', 'R_hist_73',
       'R_hist_74', 'R_hist_75', 'R_hist_76', 'R_hist_77', 'R_hist_78',
       'R_hist_79', 'R_hist_80', 'R_hist_81', 'R_hist_82', 'R_hist_83',
       'R_hist_84', 'R_hist_85', 'R_hist_86', 'R_hist_87', 'R_hist_88',
       'R_hist_89', 'R_hist_90', 'R_hist_91', 'R_hist_92', 'R_hist_93',
       'R_hist_94', 'R_hist_95', 'R_hist_96', 'R_hist_97', 'R_hist_98',
       'R_hist_99', 'R_hist_100', 'R_hist_101', 'R_hist_102',
       'R_hist_103', 'R_hist_104', 'R_hist_105', 'R_hist_106',
       'R_hist_107', 'R_hist_108', 'R_hist_109', 'R_hist_110',
       'R_hist_111', 'R_hist_112', 'R_hist_113', 'R_hist_114',
       'R_hist_115', 'R_hist_116', 'R_hist_117', 'R_hist_118',
       'R_hist_119', 'R_hist_120', 'R_hist_121', 'R_hist_122',
       'R_hist_123', 'R_hist_124', 'R_hist_125', 'R_hist_126',
       'R_hist_127']

for col in col_zscore:
    col_zscore = col + '_zscore'
    df_col_seg_meta[col_zscore] = (df_col_seg_meta[col] - df_col_seg_meta[col].mean())/df_col_seg_meta[col].std(ddof=0)

In [ ]:
df_col_seg_meta.name_main

In [ ]:
len(np.unique(df_col_seg_meta.name_main))

In [ ]:
duplicate = df_col_seg_meta[df_col_seg_meta.duplicated("name_main", keep=False)]
print("Duplicate Rows :")

duplicate_sorted = duplicate.sort_values('name_main', ascending=True)
print(duplicate_sorted.name_main)
print(len(duplicate.name_main))

In [ ]:
print(df_col_seg_meta['colorway'][df_col_seg_meta['image_fileName'] == '7286aefd-fb03-4d8f-a0bd-fc5d7e25d698'])
print(df_col_seg_meta['releaseDate'][df_col_seg_meta['image_fileName'] == '7286aefd-fb03-4d8f-a0bd-fc5d7e25d698'])
print(df_col_seg_meta['colorway'][df_col_seg_meta['image_fileName'] == '5085591a-e428-4ea3-b544-3fd47f68606e'])
print(df_col_seg_meta['releaseDate'][df_col_seg_meta['image_fileName'] == '5085591a-e428-4ea3-b544-3fd47f68606e'])

## Discard duplicated images:
# 5085591a-e428-4ea3-b544-3fd47f68606e
# 052fea39-a685-48fd-899d-c48887f81505
# 1bf72993-efab-4b42-b349-9d36caa94e63
# 093f1287-bd2d-42e9-bdf4-3c937d396e3c
# 6cb3aaf0-7932-4a14-8a1a-c2fb1a856b46

In [ ]:
#

In [ ]:
## Inference: Total

In [ ]:
df_col_seg_meta['releaseYear'] = df_col_seg_meta.releaseDate.str.split("-", n=1, expand=True)[0]
# df_col_seg_meta['brand']

In [ ]:
df_col_seg_meta['releaseYear'] = df_col_seg_meta['releaseYear'].replace(['1/1/05','1/1/12','10/3/12','11/28/12','12/8/12','4/16/12','5/10/17','5/4/13','6/24/17','6/28/17','6/30/17','7/15/17','7/2/17','7/6/17','8/5/17','9/30/17'],
                                                                        ['2005','2012','2012','2012','2012','2012','2017','2013','2017','2017','2017','2017','2017','2017','2017','2017'])

In [ ]:
print(np.unique(df_col_seg_meta['releaseYear'].values))
print(len(np.unique(df_col_seg_meta['releaseYear'].values)))

In [ ]:
i = 0
for val in df_col_seg_meta:
    print(i, val)
    i += 1
    
print('\n', "Total # of features:", str(i))

In [ ]:
## Y-labels: usHtsDescription
# y_label = pd.concat([df_col_seg_meta.iloc[:, 0], df_col_seg_meta.iloc[:, 424]], axis=1)  # gender; primaryCategory
# label_no = len(np.unique(y_label.iloc[:, 1].values))


## Y-labels: maxResell_premium
# y_label = pd.concat([df_maxResell_premium.iloc[:, 2], df_maxResell_premium.iloc[:, 0]], axis=1)
# print(y_label.head())


## Y-labels: df_bin_premium
y_label = pd.concat([df_bin_premium.iloc[:, 2], df_bin_premium.iloc[:, 4]], axis=1)
label_no = len(np.unique(y_label.iloc[:, 1].values)); print(label_no)


## Y-labels: gender
# y_label = pd.concat([df_col_seg_meta.iloc[:, 0], df_col_seg_meta.iloc[:, 420]], axis=1)
# label_no = len(np.unique(y_label.iloc[:, 1].values)); print(label_no)


# Features
color_par = pd.concat([df_col_seg_meta.iloc[:, 0], df_col_seg_meta.iloc[:, 509:521]], axis=1)  # Dist. parameter (Total 12 dimensions) == 509 ~ 520
color_ent = pd.concat([df_col_seg_meta.iloc[:, 0], df_col_seg_meta.iloc[:, 521:528]], axis=1)  # Color Entropy (7D) == 521 ~ 527
color_seg = pd.concat([df_col_seg_meta.iloc[:, 0], df_col_seg_meta.iloc[:, 528:533]], axis=1)  # Segmentation (5D) == 528 ~ 532
color_hst = pd.concat([df_col_seg_meta.iloc[:, 0], df_col_seg_meta.iloc[:, 533:917]], axis=1)  # Color Histogram (128 bin, 384D) == 533 ~ 916

In [ ]:
y_label.loc[y_label['gender']=='men','gender'] = 0
y_label.loc[y_label['gender']=='women','gender'] = 1
y_label.loc[y_label['gender']=='child','gender'] = 2
y_label.loc[y_label['gender']=='preschool','gender'] = 3
y_label.loc[y_label['gender']=='toddler','gender'] = 4

# men          3000 => 0
# women        2344 => 1
# child        1460 => 2
# preschool     420 => 3
# toddler       340 => 4

In [ ]:
y_label.loc[y_label['bin_premium']=='premium_lower','bin_premium'] = 0  # Low premium: 0
y_label.loc[y_label['bin_premium']=='premium_upper','bin_premium'] = 1  # High premium: 1

In [ ]:
print(label_no)
print(np.unique(y_label['primaryCategory'].values))
y_label.head()

In [ ]:
dict_label = {}

for i in range(0,label_no):
    dict_label[i] = np.unique(y_label['primaryCategory'].values)[i]

In [ ]:
y_label.loc[y_label['primaryCategory']=='Air Force','primaryCategory'] = 0  # Air Force: 0
y_label.loc[y_label['primaryCategory']=='Air Jordan','primaryCategory'] = 1  # Air Jordan: 1
y_label.loc[y_label['primaryCategory']=='Air Max','primaryCategory'] = 2  # Air Max: 2
y_label.loc[y_label['primaryCategory']=='Kobe','primaryCategory'] = 3  # Kobe: 3
y_label.loc[y_label['primaryCategory']=='LeBron','primaryCategory'] = 4  # LeBron: 4
y_label.loc[y_label['primaryCategory']=='Nike Basketball','primaryCategory'] = 5  # Nike Basketball: 5
y_label.loc[y_label['primaryCategory']=='Nike SB','primaryCategory'] = 6  # Nike SB: 6
y_label.loc[y_label['primaryCategory']=='adidas','primaryCategory'] = 7  # adidas: 7

np.unique(y_label['primaryCategory'].values)

In [ ]:
## Make the total dataframe

In [ ]:
df_shape_only = df_shape_only.rename(columns={'modelId': 'image_fileName'})
y_label = y_label.rename(columns={'modelId': 'image_fileName'})

df_shape_total = pd.merge(df_shape_only, y_label, on="image_fileName")

In [ ]:
df_shape_total.shape

In [ ]:
## Extract color features:

In [ ]:
color_par_total = pd.merge(color_par, y_label, on="image_fileName")
color_ent_total = pd.merge(color_ent, y_label, on="image_fileName")
color_seg_total = pd.merge(color_seg, y_label, on="image_fileName")
color_hst_total = pd.merge(color_hst, y_label, on="image_fileName")

In [ ]:
df_concat_col_seg  = pd.merge(color_ent_total, color_seg_total, on="image_fileName")  # color_hst_total; color_ent_total
del df_concat_col_seg['bin_premium_x']  # gender_x; primaryCategory_x; bin_premium_x; gender_x; inflated_profit_x

In [ ]:
df_concat_col_seg.shape

In [ ]:
df_concat_col_shape  = pd.merge(color_ent_total, df_shape_total, on="image_fileName")  # color_hst_total; color_ent_total
del df_concat_col_shape['bin_premium_x']  # gender_x; primaryCategory_x; bin_premium_x; gender_x

In [ ]:
df_concat_col_shape.shape

In [ ]:
df_concat_shape_seg  = pd.merge(df_shape_total, color_seg_total, on="image_fileName")
del df_concat_shape_seg['bin_premium_x']  # gender_x; primaryCategory_x; bin_premium_x; gender_x

In [ ]:
df_concat_shape_seg.shape

In [ ]:
df_concat_col_shape_seg  = pd.merge(df_concat_col_shape, color_seg_total, on="image_fileName")
del df_concat_col_shape_seg['bin_premium_y']  # gender_y; primaryCategory_y; bin_premium_y; gender_y

In [ ]:
df_concat_col_shape_seg.shape

# Training: Regression & XGBoost

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

In [ ]:
def classifying_labels_reg(clf_type, feature_type, loss_term):
    if clf_type == 'binary':
        pass
    elif clf_type == 'multiclass':
        model = LogisticRegression(C=1e9, multi_class='multinomial', solver='newton-cg', class_weight='balanced')
        pass

    average_str = 'weighted'
    avg_scores = []; f1_scores = []; auc_scores = []; rmse_scores = []
    cof_matricies_test = []; cof_matricies_pred = []; folds_stats_df = pd.DataFrame()
        
    # split data into train and test sets
    seed = 7; test_size = 0.20; val_size = 0.25  # 0.25 x 0.80 = 0.20    
    X_train, X_test, yy_train, yy_test = train_test_split(feature_type.iloc[:, 1:-1].astype("float64"), feature_type.iloc[:, -1], test_size=test_size, random_state=seed)
    X_train, X_val, yy_train, yy_val = train_test_split(X_train, yy_train, test_size=val_size, random_state=seed)
    print(X_train.shape); print(X_val.shape); print(X_test.shape)

    # encode string class values as integers
    label_encoder = LabelEncoder()
    label_encoder = label_encoder.fit(yy_train)
    y_train = label_encoder.transform(yy_train)
    print("y_train shape:: ", y_train.shape)    

    label_encoder = LabelEncoder()
    label_encoder = label_encoder.fit(yy_val)
    y_val = label_encoder.transform(yy_val)
    print("y_validation shape:: ", y_val.shape)
        
    label_encoder = LabelEncoder()
    label_encoder = label_encoder.fit(yy_test)
    y_test = label_encoder.transform(yy_test)
    print("y_test shape:: ", y_test.shape)    
    
    val_set = [(X_train, y_train), (X_val, y_val)]
    model.fit(X_train, y_train)
    pyplot.show()        
    
    print("====================")
    print("Fitted model:", model)
    print("====================")
    
    # make predictions for test data:
    y_pred = model.predict(X_test)
    predictions = [round(value) for value in y_pred]
    # y_preds_probs = clf.predict_proba(X_test)
    
    '''
    # retrieve performance metrics
    results = model.evals_result()
    epochs = len(results['validation_0'][loss_term])
    x_axis = range(0, epochs)
    
    # plot log loss
    fig, ax = pyplot.subplots()
    ax.plot(x_axis, results['validation_0'][loss_term], label='Train')
    ax.plot(x_axis, results['validation_1'][loss_term], label='Validation')
    ax.legend()
    pyplot.ylabel('Log Loss')
    pyplot.title('XGBoost Log Loss')
    pyplot.show()
    '''
        
    # evaluate predictions:
    accuracy = accuracy_score(y_test, predictions)
    print("Accuracy: %.2f%%" % (accuracy * 100.0))
    # r2 = r2_score(y_test, y_pred)
    # r2_adj =1- (1-r2)*(len(y_pred)-1)/(len(y_pred)-(len(model.coef_)+1))
    cnf_matrix = confusion_matrix(y_test, predictions)
    accuracy = accuracy_score(y_test, predictions)
    cohen_kappa = cohen_kappa_score(y_test, predictions)
    
    for k in y_test: cof_matricies_test.append(k)
    for i in predictions: cof_matricies_pred.append(i)
    f1score_ = f1_score(y_test, predictions, average=average_str)

    counter = 1
    folds_stats_df.at[counter, 'accuracy'] = accuracy_score(y_test, predictions)
    folds_stats_df.at[counter, 'precision'] = precision_score(y_test, predictions, average=average_str)
    folds_stats_df.at[counter, 'recall'] = recall_score(y_test, predictions, average=average_str)
    folds_stats_df.at[counter, 'f1_score'] = f1_score(y_test, predictions, average=average_str)
    folds_stats_df.at[counter, 'cohen_kappa'] = cohen_kappa_score(y_test, predictions)
    
    print("[Test] Unique labels:", np.unique(np.array(y_test)))
    print("[Pred] Unique labels:",np.unique(np.array(predictions)))
    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    print("RMSE: %f" % (rmse))
    print(folds_stats_df)
    print("====================")

    try:        
        num_y_test = np.unique(np.array(y_test), axis=0)
        num_y_test = num_y_test.shape[0]
        encoding_y_test = np.eye(num_y_test)[np.array(y_test)]
        num_predictions = np.unique(np.array(predictions), axis=0)
        num_predictions = num_predictions.shape[0]
        encoding_predictions = np.eye(num_predictions)[np.array(predictions)]        
        
        # auc_ = roc_auc_score(y_test, predictions, average=average_str)  # for binary_class
        auc_ = roc_auc_score(encoding_y_test, encoding_predictions, multi_class='ovr')  # for multi_class: 'ovo' or 'ovr'
        folds_stats_df.at[counter, 'auc'] = auc_
        auc_scores.append(auc_)

    except ValueError as e:
        logging.error(e)
        pass

    counter += 1

    avg_scores.append(accuracy)
    f1_scores.append(f1score_)
    rmse_scores.append(rmse)

    cof_matricies_test = pd.Series(cof_matricies_test)
    cof_matricies_pred = np.asarray(cof_matricies_pred)
    overall_confusion_matrix = pd.crosstab(cof_matricies_test, cof_matricies_pred, rownames=['True'], colnames=['Predicted'], margins=True)
    target_feature = get_df_name(feature_type)
    
    print("\n")
    print(f"Feature type: {target_feature} (see below table)")
    print(classification_report(cof_matricies_test, cof_matricies_pred, digits=3))
    print(avg_scores)
    print("AVerage Accuray: %f" % np.mean(avg_scores))
    print("AVerage F1: %f" % np.mean(f1_scores))
    print("AVerage AUC: %f" % np.mean(auc_scores))
    print("AVerage RMSE: %f" % np.mean(rmse_scores))

In [ ]:
def classifying_labels_XGB(clf_type, feature_type, loss_term):
    if clf_type == 'binary':
        # model = xgb.XGBRegressor(objective ='binary:logistic', colsample_bytree = 0.3, learning_rate = 0.1, max_depth = 5, alpha = 10, n_estimators = 10)
        # model = RandomForestClassifier(n_estimators=100, oob_score=True, random_state=123456)
        # model = LogisticRegression(C=1e9, solver='liblinear', class_weight='balanced')
        pass
    elif clf_type == 'multiclass':
        model = xgb.XGBRegressor(objective ='multi:softmax', colsample_bytree = 0.3, learning_rate = 0.05, max_depth = 5, alpha = 10, n_jobs=50, n_estimators = 100, num_class = label_no)
        #model = RandomForestClassifier(n_estimators=100, oob_score=True, random_state=123456)
        #model = LogisticRegression(C=1e9, multi_class='multinomial', solver='newton-cg', class_weight='balanced')
        pass

    average_str = 'weighted'
    avg_scores = []; f1_scores = []; auc_scores = []; rmse_scores = []
    cof_matricies_test = []; cof_matricies_pred = []; folds_stats_df = pd.DataFrame()
        
    # split data into train and test sets
    seed = 7; test_size = 0.20; val_size = 0.25  # 0.25 x 0.80 = 0.20    
    X_train, X_test, yy_train, yy_test = train_test_split(feature_type.iloc[:, 1:-1].astype("float64"), feature_type.iloc[:, -1], test_size=test_size, random_state=seed)
    X_train, X_val, yy_train, yy_val = train_test_split(X_train, yy_train, test_size=val_size, random_state=seed)
    print("X_train shape:",X_train.shape); print("X_validation shape:",X_val.shape); print("X_test shape:",X_test.shape)

    # encode string class values as integers
    label_encoder = LabelEncoder()
    label_encoder = label_encoder.fit(yy_train)
    y_train = label_encoder.transform(yy_train)
    print("y_train shape:", y_train.shape)    

    label_encoder = LabelEncoder()
    label_encoder = label_encoder.fit(yy_val)
    y_val = label_encoder.transform(yy_val)
    print("y_validation shape:", y_val.shape)
        
    label_encoder = LabelEncoder()
    label_encoder = label_encoder.fit(yy_test)
    y_test = label_encoder.transform(yy_test)
    print("y_test shape:", y_test.shape)
    
    list_original_label = []
    for k in range(0, len(np.unique(np.array(y_test)))):
        list_original_label.append(dict_label[k])
        
    val_set = [(X_train, y_train), (X_val, y_val)]
    model.fit(X_train, y_train, early_stopping_rounds=5, eval_metric=loss_term, eval_set=val_set, verbose=False)

    try:
        plot_importance(model)
        pyplot.show()
        
    except ValueError as e:
        logging.error(e)
        pass
    
    print("====================")
    print("Fitted model:", model)
    print("====================")
    
    # make predictions for test data:
    y_pred = model.predict(X_test)
    predictions = [round(value) for value in y_pred]
    # y_preds_probs = clf.predict_proba(X_test)
    
    # retrieve performance metrics
    results = model.evals_result()
    epochs = len(results['validation_0'][loss_term])
    x_axis = range(0, epochs)
    
    # plot log loss
    fig, ax = pyplot.subplots()
    ax.plot(x_axis, results['validation_0'][loss_term], label='Train')
    ax.plot(x_axis, results['validation_1'][loss_term], label='Validation')
    ax.legend()
    pyplot.ylabel('Log Loss')
    pyplot.title('XGBoost Log Loss')
    pyplot.show()
        
    # evaluate predictions:
    accuracy = accuracy_score(y_test, predictions)
    print(f"No. of Epochs: {len(x_axis)}")
    print("Accuracy: %.2f%%" % (accuracy * 100.0))
    # r2 = r2_score(y_test, y_pred)
    # r2_adj =1- (1-r2)*(len(y_pred)-1)/(len(y_pred)-(len(model.coef_)+1))
    cnf_matrix = confusion_matrix(y_test, predictions)
    accuracy = accuracy_score(y_test, predictions)
    cohen_kappa = cohen_kappa_score(y_test, predictions)
    
    for k in y_test: cof_matricies_test.append(k)
    for i in predictions: cof_matricies_pred.append(i)
    f1score_ = f1_score(y_test, predictions, average=average_str)

    counter = 1
    folds_stats_df.at[counter, 'accuracy'] = accuracy_score(y_test, predictions)
    folds_stats_df.at[counter, 'precision'] = precision_score(y_test, predictions, average=average_str)
    folds_stats_df.at[counter, 'recall'] = recall_score(y_test, predictions, average=average_str)
    folds_stats_df.at[counter, 'f1_score'] = f1_score(y_test, predictions, average=average_str)
    folds_stats_df.at[counter, 'cohen_kappa'] = cohen_kappa_score(y_test, predictions)   
    
    print("[Test] Unique labels original:", np.unique(list_original_label))
    print("[Test] Unique labels:", np.unique(np.array(y_test)))
    print("[Pred] Unique labels:", np.unique(np.array(predictions)))
    print(cnf_matrix, '\n')
    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    print(folds_stats_df)
    print("RMSE: %f" % (rmse))

    try:        
        num_y_test = np.unique(np.array(y_test), axis=0)
        num_y_test = num_y_test.shape[0]
        encoding_y_test = np.eye(num_y_test)[np.array(y_test)]
        num_predictions = np.unique(np.array(predictions), axis=0)
        num_predictions = num_predictions.shape[0]
        encoding_predictions = np.eye(num_predictions)[np.array(predictions)]        
        
        # auc_ = roc_auc_score(y_test, predictions, average=average_str)  # for binary_class
        auc_ = roc_auc_score(encoding_y_test, encoding_predictions, multi_class='ovr')  # for multi_class: 'ovo' or 'ovr'
        folds_stats_df.at[counter, 'auc'] = auc_
        auc_scores.append(auc_)

    except IndexError as e:
        logging.error(e)
        pass

    counter += 1

    avg_scores.append(accuracy)
    f1_scores.append(f1score_)
    rmse_scores.append(rmse)

    cof_matricies_test = pd.Series(cof_matricies_test)
    cof_matricies_pred = np.asarray(cof_matricies_pred)
    overall_confusion_matrix = pd.crosstab(cof_matricies_test, cof_matricies_pred, rownames=['True'], colnames=['Predicted'], margins=True)
    target_feature = get_df_name(feature_type)

    print("\n")
    print(f"Feature type: {target_feature} (see below table)")
    print(classification_report(cof_matricies_test, cof_matricies_pred, digits=3))
    print("Average Accuray: %.3f" % np.around(np.mean(avg_scores), decimals=3))
    print("Average F1: %.3f" % np.around(np.mean(f1_scores), decimals=3))
    print("Average AUC: %.3f" % np.around(np.mean(auc_scores), decimals=3))
    print("Average RMSE: %.3f" % np.around(np.mean(rmse_scores), decimals=3))
    
    # print(frame_feature_importance.T.mean(axis=0))
    # print(np.std(avg_scores))
    # print('Overall Accuracy:', accuracy_score(cof_matricies_test, cof_matricies_pred))
    # print('Overall Precision:', precision_score(cof_matricies_test, cof_matricies_pred, average=average_str))
    # print('Overall Recall:', recall_score(cof_matricies_test, cof_matricies_pred, average=average_str))
    # print('Overall F1-score:', f1_score(cof_matricies_test, cof_matricies_pred, average=average_str))
    # print('Overall AUC score:', roc_auc_score(cof_matricies_test, cof_matricies_pred, average='macro'))
    # print('Overall AUC score:', multiclass_roc_auc_score(cof_matricies_test, cof_matricies_pred))
    # cnf_matrix = confusion_matrix(cof_matricies_test, cof_matricies_pred); print(cnf_matrix)
    # print(folds_stats_df['accuracy'].mean())
    # print(r2, r2_adj)

# Execution Function

In [ ]:
def do_Classification():
    clf_type = 'multiclass'  # = 'binary'
    loss_term = 'mlogloss'
    feature_type = df_shape_total
    classifying_labels_reg(clf_type, feature_type, loss_term)
    #classifying_labels_XGB(clf_type, feature_type, loss_term)

# Solo: color_par_total; color_ent_total; color_seg_total; color_hst_total; df_shape_total    
# Concat: df_concat_col_seg; df_concat_col_shape; df_concat_shape_seg; df_concat_col_shape_seg

In [ ]:
if __name__ == '__main__':
    do_Classification()
    print('\n'+str("***** All successfully done *****")+'\n')

# Training: Multi-Layer Perceptron (Classifier)

In [ ]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split

import pandas as pd

In [ ]:
# Solo: color_par_total; color_ent_total; color_hst_total; color_seg_total; df_shape_total    
# Concat: df_concat_col_seg; df_concat_col_shape; df_concat_shape_seg; df_concat_col_shape_seg

seed = 7
test_size = 0.20
target_dataframe = color_seg_total

train_XX, test_XX, train_YY, test_YY = train_test_split(target_dataframe.iloc[:,1: -1].astype("float64"), 
                                                        target_dataframe.iloc[:, -1].astype("int64"),
                                                        test_size=test_size, random_state=seed)

train_X = train_XX.to_numpy()  # 넘파이 배열로 만들기
test_X  = test_XX.to_numpy()   # 넘파이 배열로 만들기
train_Y = train_YY.to_numpy().reshape((-1,1))  # 데이터프레임 형태의 타겟값을 넘파이 배열로 만들기
test_Y = test_YY.to_numpy().reshape((-1,1))    # 데이터프레임 형태의 타겟값을 넘파이 배열로 만들기

# 데이터를 텐서 형태로 변환
train_X = torch.from_numpy(train_X).float()
train_Y = torch.from_numpy(train_Y).long()

test_X = torch.from_numpy(test_X).float()
test_Y = torch.from_numpy(test_Y).float()

trainset = TensorDataset(train_X, train_Y)
testset = TensorDataset(test_X, test_Y)

In [ ]:
print(trainset[2])  # Just peep one example
# classes = ('Air Force','Air Jordan','Air Max','Kobe','LeBron','Nike Basketball','Nike SB','adidas')
classes = ('Low Premium','High Premium')

train_loader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2, drop_last=True)
test_loader  = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2, drop_last=True)

In [ ]:
print(train_X.shape)
print(train_Y.shape)
print(test_X.shape)
print(test_Y.shape)

In [ ]:
# 12005/4  # batch_no=4

In [ ]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
#         self.fc1 = nn.Linear(524, 96)  # reference
#         self.fc2 = nn.Linear(96, 96)
#         self.fc3 = nn.Linear(96, 96)
#         self.fc4 = nn.Linear(96, 96)
#         self.fc5 = nn.Linear(96, 96)
#         self.fc6 = nn.Linear(96, 8)    # 8; 2

#         self.fc1 = nn.Linear(12, 7)
#         self.fc2 = nn.Linear(7, 7)
#         self.fc3 = nn.Linear(7, 7)
#         self.fc4 = nn.Linear(7, 2)

#         self.fc1 = nn.Linear(7, 4)
#         self.fc2 = nn.Linear(4, 4)
#         self.fc3 = nn.Linear(4, 4)
#         self.fc4 = nn.Linear(4, 2)

        self.fc1 = nn.Linear(5, 3)
        self.fc2 = nn.Linear(3, 3)
        self.fc3 = nn.Linear(3, 3)
        self.fc4 = nn.Linear(3, 2)

#         self.fc1 = nn.Linear(384, 96)  # 384; 512
#         self.fc2 = nn.Linear(96, 96)
#         self.fc3 = nn.Linear(96, 96)
#         self.fc4 = nn.Linear(96, 96)
#         self.fc5 = nn.Linear(96, 96)
#         self.fc6 = nn.Linear(96, 2)

#         self.fc1 = nn.Linear(1536, 96)
#         self.fc2 = nn.Linear(96, 96)
#         self.fc3 = nn.Linear(96, 96)
#         self.fc4 = nn.Linear(96, 96)
#         self.fc5 = nn.Linear(96, 96)
#         self.fc6 = nn.Linear(96, 96)
#         self.fc7 = nn.Linear(96, 2)

    def forward(self, x):
#         x = torch.flatten(x, 1)  # 배치를 제외한 모든 차원을 평탄화(flatten)

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)        

#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = F.relu(self.fc3(x))
#         x = F.relu(self.fc4(x))
#         x = F.relu(self.fc5(x))
#         #x = F.relu(self.fc6(x))
#         x = self.fc6(x)
                
#       return F.log_softmax(x)  # nn.CrossEntropyLoss() 에서 같은 작업 수행
        return x

model = Net()  # It should be run for every new data!

In [ ]:
# 오차함수
criterion = nn.CrossEntropyLoss()

# 최적화 담당
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# 학습 실시
list_loss = []
for epoch in range(300):  # 데이터셋 학습 N차례 (e.g., 300회) 반복
    running_loss = 0.0
    
    for i, data in enumerate(train_loader, 0):
        # [inputs, labels]의 목록인 data로부터 입력을 받은 후,
        inputs, labels = data

        # 경사 초기화: 변화도(Gradient) 매개변수를 0으로 만들고,
        optimizer.zero_grad()

        # 순전파 + 역전파 + 최적화를 한 후,
        outputs = model(inputs)            # 순전파
        labels = labels.squeeze_()         # https://stackoverflow.com/questions/49206550/pytorch-error-multi-target-not-supported-in-crossentropyloss/49209628
        loss = criterion(outputs, labels)  # 오차
        loss.backward()                    # 역전파 계산
        optimizer.step()                   # 가중치 업데이트

        running_loss += loss.item()        # 총 오차 업데이트
        
        # 통계 출력.
        report_tick = 10
        if i % report_tick == report_tick-1:   # print every "report_tick" mini-batches
            print('[%d, %5d] loss: %.5f' %
                 (epoch + 1, i + 1, running_loss / report_tick))
            list_loss.append(running_loss)
            running_loss = 0.0

print('\n','Finished Training')

In [ ]:
import matplotlib.pyplot as plt

plt.plot(list_loss)
plt.title('Loss')
plt.xlabel('1K Mini-batch')
plt.show()

In [ ]:
# 테스트 데이터를 집어 넣는다.
test_x, test_y = Variable(test_X), Variable(test_Y)

# 테스트 데이터를 집어 넣어서 학습 시킨 다음, max 값(확률이 더 높은 값)을 출력한다.
result = torch.max(model(test_x).data, 1)[1]

# 정확도 계산
accuracy = sum(test_y.data.numpy() == result.numpy().reshape((-1,1))) / len(test_y.data.numpy())
print(accuracy.mean())

In [ ]:
from sklearn.metrics import f1_score
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html

print(f1_score(test_y.data.numpy(), result.numpy().reshape((-1,1)), average='macro'))
print(f1_score(test_y.data.numpy(), result.numpy().reshape((-1,1)), average='micro'))
print(f1_score(test_y.data.numpy(), result.numpy().reshape((-1,1)), average='weighted'))  # Using this one

In [ ]:
from sklearn.metrics import cohen_kappa_score

cohen_kappa_score(test_y.data.numpy(), result.numpy().reshape((-1,1)))

In [ ]:
from sklearn.metrics import precision_score
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html

print(precision_score(test_y.data.numpy(), result.numpy().reshape((-1,1)), average='macro'))
print(precision_score(test_y.data.numpy(), result.numpy().reshape((-1,1)), average='micro'))
print(precision_score(test_y.data.numpy(), result.numpy().reshape((-1,1)), average='weighted'))  # Using this one

In [ ]:
from sklearn.metrics import recall_score

print(recall_score(test_y.data.numpy(), result.numpy().reshape((-1,1)), average='macro'))
print(recall_score(test_y.data.numpy(), result.numpy().reshape((-1,1)), average='micro'))
print(recall_score(test_y.data.numpy(), result.numpy().reshape((-1,1)), average='weighted'))  # Using this one

In [ ]:
# 학습한 모델 저장

PATH = '_results/mlp_total.pth'
torch.save(model.state_dict(), PATH)

In [ ]:
#

In [ ]:
## Load the saved model:

In [ ]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(524, 96)
        self.fc2 = nn.Linear(96, 96)
        self.fc3 = nn.Linear(96, 96)
        self.fc4 = nn.Linear(96, 96)
        self.fc5 = nn.Linear(96, 96)
        self.fc6 = nn.Linear(96, 8)

    def forward(self, x):
#       x = torch.flatten(x, 1) # 배치를 제외한 모든 차원을 평탄화(flatten)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))

        x = self.fc6(x)        
#       return F.log_softmax(x)
        return x

net = Net()

In [ ]:
PATH = '_results/mlp_total.pth'  # 저장한 모델 로딩
net.load_state_dict(torch.load(PATH))

In [ ]:
dataiter = iter(test_loader)
feats, labels = dataiter.next()
outputs = net(feats)

In [ ]:
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]] for j in range(4)))

In [ ]:
correct = 0
total = 0

# 학습 중이 아니므로, 출력에 대한 변화도를 계산할 필요 X
with torch.no_grad():
    for data in test_loader:
        feats, labels = data
        
        # 신경망에 테스트 데이터를 통과시켜 출력을 계산합니다
        outputs = net(feats)
        
        # 가장 높은 값(energy)를 갖는 분류(class)를 정답으로 선택
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == torch.transpose(labels,0,1)).sum().item()        

print('Accuracy of the network on the test datatest: %d %%' % (100 * correct / total))

# Training: Multi-Layer Perceptron (Regressor)

In [ ]:
# machine learning module
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np

# ANN module
import torch
from torch import nn, optim                       # torch 제공 신경망 기술, 손실함수, 최적화를 할 수 있는 함수들을 불러온다.
from torch.utils.data import DataLoader, Dataset  # 데이터를 모델에 사용할 수 있게 정리해주는 라이브러리.
import torch.nn.functional as F                   # torch 내의 세부적인 기능을 불러옴.

# Loss
from sklearn.metrics import mean_squared_error    # regression 문제의 모델 성능 측정을 위해서 MSE를 불러온다.

import matplotlib.pyplot as plt

In [ ]:
# Solo: color_par_total; color_ent_total; color_seg_total; color_hst_total; df_shape_total    
# Concat: df_concat_col_seg; df_concat_col_shape; df_concat_shape_seg; df_concat_col_shape_seg

target_dataframe = color_hst_total

# 데이터를 넘파이 배열로 만들기
X = target_dataframe.iloc[:, 1:-1].astype("float64").to_numpy()  # 데이터프레임에서 타겟값 제외하고 넘파이 배열로 만들기
Y = target_dataframe.iloc[:, -1].to_numpy().reshape((-1,1))      # 데이터프레임 형태의 타겟값을 넘파이 배열로 만들기

# 데이터 스케일링: sklearn 제공 MinMaxScaler => (X-min(X))/(max(X)-min(X)) 계산
scaler = MinMaxScaler() 
scaler.fit(X) 
X = scaler.transform(X)

scaler.fit(Y)
Y = scaler.transform(Y)

In [ ]:
i = 0
for val in target_dataframe:
    print(i, val)
    i += 1
    
print('\n', "Total # of features:", str(i))

In [ ]:
# torch의 Dataset 을 상속.
class TensorData(Dataset):

    def __init__(self, x_data, y_data):
        self.x_data = torch.FloatTensor(x_data)
        self.y_data = torch.FloatTensor(y_data)
        self.len = self.y_data.shape[0]

    def __getitem__(self, index):

        return self.x_data[index], self.y_data[index] 

    def __len__(self):
        return self.len

In [ ]:
# 전체 데이터를 학습 데이터와 평가 데이터로 나눈다.
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

# 학습 데이터, 시험 데이터 배치 형태로 구축하기
trainsets = TensorData(X_train, Y_train)
trainloader = torch.utils.data.DataLoader(trainsets, batch_size=32, shuffle=True)  # batch_size=4;32;64

testsets = TensorData(X_test, Y_test)
testloader = torch.utils.data.DataLoader(testsets, batch_size=32, shuffle=False)  # batch_size=4;32;64

In [ ]:
print(X.shape)
print(Y.shape)
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

In [ ]:
# 모델 구축: for small dimensions
class Regressor(nn.Module):
    def __init__(self):
        super().__init__() # 모델 연산 정의
        self.fc1 = nn.Linear(5, 3, bias=True)
        self.fc2 = nn.Linear(3, 3, bias=True)
        self.fc3 = nn.Linear(3, 3, bias=True) 
        self.fc4 = nn.Linear(3, 1, bias=True)
        self.dropout = nn.Dropout(0.2) # 연산이 될 때마다 20%의 비율로 랜덤하게 노드를 없앤다.

    def forward(self, x): # 모델 연산의 순서를 정의
        x = F.relu(self.fc1(x)) # Linear 계산 후 활성화 함수 ReLU를 적용한다.
        x = self.dropout(F.relu(self.fc2(x))) # 은닉층2에서 드랍아웃을 적용한다.
        x = self.dropout(F.relu(self.fc3(x))) 
        x = F.relu(self.fc4(x)) # Linear 계산 후 활성화 함수 ReLU를 적용한다.
      
        return x
    
## 주의 사항:
# 드랍아웃은 과적합(overfitting)을 방지하기 위해 노드의 일부를 배제하고 계산하는 방식이기 때문에 절대로 출력층에 사용해서는 안 된다.

In [ ]:
# 모델 구축
class Regressor(nn.Module):
    def __init__(self):
        super().__init__() # 모델 연산 정의
        self.fc1 = nn.Linear(X.shape[1], 96, bias=True) # 입력층(524) -> 은닉층1(96)으로 가는 연산
        self.fc2 = nn.Linear(96, 96, bias=True) # 은닉층1(96) -> 은닉층2(96)으로 가는 연산
        self.fc3 = nn.Linear(96, 96, bias=True) 
        self.fc4 = nn.Linear(96, 96, bias=True) 
        self.fc5 = nn.Linear(96, 96, bias=True)         
        self.fc6 = nn.Linear(96, 1, bias=True) # 은닉층5(96) -> 출력층(1)으로 가는 연산
        self.dropout = nn.Dropout(0.2) # 연산이 될 때마다 20%의 비율로 랜덤하게 노드를 없앤다.

    def forward(self, x): # 모델 연산의 순서를 정의
        x = F.relu(self.fc1(x)) # Linear 계산 후 활성화 함수 ReLU를 적용한다.
        x = self.dropout(F.relu(self.fc2(x))) # 은닉층2에서 드랍아웃을 적용한다.(즉, 30개의 20%인 6개의 노드가 계산에서 제외된다.)
        x = self.dropout(F.relu(self.fc3(x))) 
        x = self.dropout(F.relu(self.fc4(x))) 
        x = self.dropout(F.relu(self.fc5(x)))
        x = F.relu(self.fc6(x)) # Linear 계산 후 활성화 함수 ReLU를 적용한다.
      
        return x
    
## 주의 사항:
# 드랍아웃은 과적합(overfitting)을 방지하기 위해 노드의 일부를 배제하고 계산하는 방식이기 때문에 절대로 출력층에 사용해서는 안 된다.

In [ ]:
model = Regressor()
criterion = nn.MSELoss()

optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-7)  # weight_decay: L2 norm 정규화에서 사용하는 penalty

In [ ]:
loss_ = [] # loss를 저장할 리스트.
n = len(trainloader)

for epoch in range(300):  # 300; 400
    running_loss = 0.0    # 한 에폭이 돌 때 그안에서 배치마다 loss가 나온다. 
                          # 즉 한번 학습할 때 그렇게 쪼개지면서 loss가 다 나오니 MSE를 구하기 위해서 사용한다.

    for i, data in enumerate(trainloader, 0): # 무작위로 섞인 32개의 데이터가 담긴 배치가 하나씩 들어온다.
    
        inputs, values = data # data에는 X, Y가 들어있다.
        optimizer.zero_grad() # 최적화 초기화.

        outputs = model(inputs) # 모델에 입력값을 넣어 예측값을 산출한다.
        loss = criterion(outputs, values) # 손실함수를 계산. error 계산.
        loss.backward() # 손실 함수를 기준으로 역전파를 설정한다.
        optimizer.step() # 역전파를 진행하고 가중치를 업데이트한다.

        running_loss += loss.item() # 총 오차 업데이트: epoch 마다 평균 loss를 계산하기 위해 배치 loss를 더한다.
        
        # 통계 출력:
        report_tick = 10
        if i % report_tick == report_tick-1:   # print every 10 mini-batches
            print('[%d, %5d] loss: %.3f' %
                 (epoch + 1, i + 1, running_loss / report_tick))
  
    loss_.append(running_loss/n) # MSE(Mean Squared Error) 계산

print('\n','Finished Training')

In [ ]:
plt.plot(loss_)
plt.title('Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
def evaluation(dataloader):

    predictions = torch.tensor([], dtype=torch.float) # 예측값을 저장하는 텐서.
    actual = torch.tensor([], dtype=torch.float) # 실제값을 저장하는 텐서.

    with torch.no_grad():
        model.eval() # 평가를 할 땐 반드시 eval()을 사용해야 한다.

        for data in dataloader:
            inputs, values = data
            outputs = model(inputs)

            predictions = torch.cat((predictions, outputs), 0) # cat함수를 통해 예측값을 누적.
            actual = torch.cat((actual, values), 0) # cat함수를 통해 실제값을 누적.

    predictions = predictions.numpy() # 넘파이 배열로 변경.
    actual = actual.numpy() # 넘파이 배열로 변경.
    rmse = np.sqrt(mean_squared_error(predictions, actual)) # sklearn을 이용해 RMSE를 계산.
    r_sq_score = r2_score(predictions, actual)
    mae = mean_absolute_error(predictions, actual)

    return r_sq_score, rmse, mae

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

train_rmse = evaluation(trainloader)
test_rmse = evaluation(testloader)

print(f'[Train] R^2, RMSE, MAE:{train_rmse}')
print(f'[Test] R^2, RMSE, MAE:{test_rmse}')

In [ ]:
## buffer

#### End.